1. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk sorszámozott listát a szálláshelyek adatairól!&nbsp;</span> 

a. A lista legyen a szálláshely típusa szerint, azon belül hely szerint, majd a neve szerint növekvően rendezve! 

b. A számozás típusonként kezdődjön újra!

In [1]:
Select  row_number() over(
            PARTITION BY tipus
) as 'sorszám',
        *
from szallashely

(21 rows affected)

Total execution time: 00:00:00.045

sorszám,SZALLAS_ID,SZALLAS_NEV,HELY,CSILLAGOK_SZAMA,TIPUS,ROGZITETTE,ROGZ_IDO,CIM
1,7,Szieszta Apartmanház,Balaton-dél,0,Apartman,Mari,2016-03-11,"8630 Balatonboglár, Dózsa György utca 108"
2,18,Fortuna Apartman,Dél-Somogy,0,Apartman,Béla,2016-04-07,"7570 Barcs, Kossuth u. 13."
3,3,Fortuna Apartman,Hajdúbihar megye,0,Apartman,Sára,2016-03-06,"4200 Hajdúszoboszló, Wesselényi utca 56."
1,20,Kollégium,Budapest,0,Diákszálló,Sára,2016-04-07,"1146 Budapest, Ajtósi Dürer sor 23."
1,8,Hotel Három Hattyú,Balaton-dél,4,Hotel,Béla,2016-03-13,"8623 Balatonföldvár, Rákóczi Ferenc út 45."
2,6,Kentaur Hotel,Balaton-dél,3,Hotel,Béla,2016-03-08,"8600 Siófok (Széplak), Akácfa utca 1"
3,12,Gold Hotel,Budapest,3,Hotel,Béla,2016-03-24,"1016 Budapest, Hegyalja út 12"
4,10,Lagúna Hotel,Budapest,4,Hotel,Sára,2016-03-20,"1097 Budapest, Albert Flórián út 3"
5,16,Bagoly Hotel,Pest megye,3,Hotel,Mari,2016-04-01,"2230 Gyömrő, Bergszász u. 5."
6,17,Szarvas Hotel,Tolna megye,4,Hotel,Béla,2016-04-04,"7121 Szálka, Petőfi Sándor u."


2. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk lekérdezést, amely a foglalások adatait jeleníti meg!&nbsp;</span> 

a. A lista tartalmazzon két új oszlopot, amelyek az adott ügyfél legrövidebb, illetve leghosszabb foglalásának hosszát (a foglalt napok számát) mutatják meg! 

b. Az oszlopokat nevezzük el értelemszerűen!

```
3. Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, valamint azt, hogy a foglalás összesen hány főre történt! 
```

a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző három foglalása összesen hány főre történt!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

4. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk listát arról, hogy az egyes szálláshelyeken évente hány foglalás történt!&nbsp;</span> 

a. A lista jelenítse meg a szálláshely azonosítóját, nevét, az évet, és a foglalások számát!

b. Egy új oszlopban jelenítsük meg a szálláshely eddigi éves foglalásainak számát is (az aktuálisat is beleértve). 

c. Az évnél a METTOL dátumot vegyük figyelembe! 

d. A lista ne tartalmazzon duplikált sorokat!

In [2]:
SELECT distinct szh.SZALLAS_ID,YEAR(f.METTOL), count(*) over(partition by szh.SZALLAS_ID, YEAR(f.METTOL)),    
                -- eddigi foglalásk száma
                COUNT(*) over(partition by szh.szallas_id, year(f.METTOL)
                                order by f.mettol
                                range between unbounded preceding AND current ROW)
FROM Szallashely szh join szoba sz on szh.SZALLAS_ID=sz.SZALLAS_FK
                    join Foglalas f on sz.SZOBA_ID=f.SZOBA_FK

(811 rows affected)

Total execution time: 00:00:00.036

SZALLAS_ID,(No column name),(No column name),(No column name)
3,2016,35,31
6,2016,150,44
6,2017,29,20
8,2016,157,80
12,2016,152,12
8,2016,157,32
10,2017,20,12
2,2016,13,7
7,2016,28,23
12,2017,27,18


5. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsük listát a szálláshelyekről! A lista tartalmazza a szálláshely azonosítóját, nevét, és a csillagok számát csillagszám szerint csökkenő, azon belül szállásnév szerint növekvő sorrendben!&nbsp;</span> 

a. Vegyünk fel két új oszlopot, amely a sorrend szerint előző szálláshely nevét, illetve azonosítóját is megjeleníti!

6. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon termékeket, amelyekből az átlagosnál kevesebb van raktáron!&nbsp;</span> 

a. Csak a termék kódja és megnevezése jelenjen meg!

In [5]:
SELECT TERMEKKOD, MEGYS
From Termek
WHERE KESZLET < (
    Select avg(KESZLET)
    from Termek
)

(437 rows affected)

Total execution time: 00:00:00.007

TERMEKKOD,MEGYS
01010110T,db
01010304T,db
01020300T,db
01030091T,db
01030092T,db
01040307T,ív
01040502T,db
01080093T,db
01080512T,doboz
01080513T,ív


7. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon ügyfeleket, akik még nem fizettek bankkártyával!&nbsp;</span> 

a. Csak az ügyfelek neve jelenjen meg!

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'Ugyfel'.

Total execution time: 00:00:00.003

8. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Mennyibe kerül a második legnagyobb listaárú termék?</span>

9. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyik raktár(ak)ban van az a termék, amelyből a legnagyobb készlet van?&nbsp;</span> 

a. Csak a raktár(ak) neve jelenjen meg!

10. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyek azok a termékek, amelyek listaára nagyobb, mint bármelyik olyan terméké, amelynek nevében az Ajándék szó szerepel?&nbsp;</span> 

a. A termékek kódját és nevét is jelenítsük meg!

11. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyik hónapokban adtak le az ügyfelek több rendelést, mint januárban?&nbsp;</span> 

a. Elég a hónapok sorszámait megjeleníteni. 

b. A listát rendezzük a hónapok száma szerint növekvő sorrendbe!

12. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyek azok a termékek, amelyek listaára az átlagostól max. 20%-kal tér el?&nbsp;</span> 

a. A termék kódját és nevét is jelenítsük meg!

13. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk listát (sorszámonként) az egyes rendelések teljes összegéről!&nbsp;</span> 

a. Szűrjünk azon tételekre, ahol ez az érték nagyobb, mint Viktor Éva rendeléseinek teljes összege! 

b. A teljes összeg oszlopot nevezzük el értelemszerűen!

14. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azokat a termékkategóriákat, amelyből 2017 januárban nem történt rendelés!&nbsp;</span> 

a. Elég a kategóriák nevét megjeleníteni!

15. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon rendelési tételeket, amelynek értéke nagyobb, mint a legdrágább termék listaára azon termékek közül, amelyek szülő kategóriája a ruha!</span>